# **APD Frame Similarity Assessment - Detailed Process Explanation**

# **Step 1: Import Libraries**
# Import the necessary libraries for video processing (OpenCV), numerical operations (NumPy), time tracking (time),
# and memory profiling (psutil) to evaluate memory usage before and after processing the video.

# **Step 2: Absolute Pixel Difference - Frame Similarity Assessment**
# The `calculate_frame_difference` function calculates the absolute difference between two frames.
# It then thresholds this difference to ignore insignificant changes, counting the number of pixels that changed significantly.
# The motion similarity is determined by comparing the change ratio to a predefined threshold.
# This function helps in evaluating if the current frame is similar to the previous one based on motion changes.

# **Step 3: Frame Similarity Check**
# The `is_frame_similar` function compares the current frame with the previous one.
# If the difference is below a defined threshold, indicating high similarity, the frame is skipped; otherwise, it is processed.
# This function ensures that redundant frames (those with minimal motion) are skipped, optimizing the video processing.

# **Step 4: Read Video Frames**
# The `get_video_frames` function reads and extracts all frames from the input video file.
# Each frame is converted to grayscale for easier processing, and the frames are stored in a list.
# This step prepares the frames for comparison and processing.

# **Step 5: Video Loading Time Calculation**
# The time taken to load the video frames is recorded using the `time.time()` function.
# This helps in evaluating the performance of the video loading step and gives an insight into how much time is spent before processing.

# **Step 6: Video Writer Setup**
# A video writer is set up using OpenCV to save the processed video with the specified codec (`mp4v`) and frame rate (30 FPS).
# The frame dimensions (height and width) are obtained from the first frame of the video, which is used to initialize the video writer.

# **Step 7: Frame Processing and Writing**
# The frames are processed one by one, starting from the second frame.
# If the current frame is similar to the last processed frame (based on motion similarity), it is skipped.
# If the frame is not similar, it is written to the output video.
# This process helps reduce video size by skipping frames that don't introduce significant changes.

# **Step 8: Processing Time Tracking**
# The time taken to process the frames is tracked by recording the start and end times.
# The total processing time includes both the time spent on comparing frames and writing them to the output video.

# **Step 9: Memory Usage Evaluation**
# Memory usage before and after processing is tracked using the `psutil` library.
# The `initial_memory` represents memory usage before processing, while `final_memory` is the memory usage after processing.
# The difference (`memory_consumed`) gives an idea of how much memory was utilized during the frame comparison and video writing process.

# **Step 10: Log Metrics**
# After processing the video, various metrics are logged:
#  - Total number of frames processed and skipped
#  - Percentage of skipped frames
#  - Frames processed per second (FPS)
#  - Time spent on loading frames, processing frames, and total time
#  - Memory usage before and after processing
# These logs help assess the efficiency of the video processing, including time and memory consumption.

# **Step 11: Final Video Writing and Resource Cleanup**
# Once all frames are processed, the video writer is released to save the final processed video to disk.
# This step ensures that resources are properly freed and the output video is saved in the desired format.


In [ ]:
# Dry Run
import cv2
import numpy as np
import time
import psutil  # For memory profiling

# Set the video path (update this to your video file path)
video_path = '/content/00067cfb-e535423e.mov'  # Change this to your video file path
output_video_path = '/content/processed_video.mp4'  # Output video path

# Function to calculate frame difference with motion thresholding
def calculate_frame_difference(frame1, frame2):
    # Calculate the absolute difference between frames
    diff = cv2.absdiff(frame1, frame2)

    # Thresholding to count significant changes
    _, diff = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)  # Ignore low-intensity changes
    non_zero_count = np.count_nonzero(diff)
    change_ratio = non_zero_count / frame1.size

    return 1 - change_ratio  # Higher values indicate lower motion

# Function to check if frames are similar based on spatial difference
def is_frame_similar(frame1, frame2, threshold=0.90):
    # Calculate motion similarity
    motion_similarity = calculate_frame_difference(frame1, frame2)

    # Check against threshold
    return motion_similarity > threshold

# Read video frames
def get_video_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
        frames.append(gray_frame)
    cap.release()
    return frames

# Load video frames
start_time = time.time()  # Start time for reading frames
frames = get_video_frames(video_path)
loading_time = time.time() - start_time  # Time taken to load frames

# Set up the video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for mp4
fps = 30  # Set to your video's frame rate
frame_height, frame_width = frames[0].shape
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Process frames and write to output video
frame_count = len(frames)
skipped_frames = 0
out.write(cv2.cvtColor(frames[0], cv2.COLOR_GRAY2BGR))  # Write the first frame
last_processed_frame_index = 0  # Track the last processed frame index

# Start time for processing
start_processing_time = time.time()

# Start comparing from frame 1 onwards
for i in range(1, frame_count):
    # Compare current frame with the last processed frame
    if is_frame_similar(frames[last_processed_frame_index], frames[i]):
        skipped_frames += 1
    else:
        out.write(cv2.cvtColor(frames[i], cv2.COLOR_GRAY2BGR))  # Write the current frame if not similar
        last_processed_frame_index = i  # Update the last processed frame index

# Calculate processing time
end_processing_time = time.time()
processing_time = end_processing_time - start_processing_time
total_time = loading_time + processing_time  # Total time includes loading and processing

# Release the video writer
out.release()

# Memory usage before and after processing
process = psutil.Process()
memory_info = process.memory_info()
initial_memory = memory_info.rss / (1024 * 1024)  # Convert bytes to MB
final_memory = (memory_info.rss + (skipped_frames * frames[0].nbytes)) / (1024 * 1024)  # Rough estimate of final memory
memory_consumed = final_memory - initial_memory

# Log the required information
print(f"Processed video saved as: {output_video_path}")
print(f"Total number of frames: {frame_count}")
print(f"Number of frames skipped: {skipped_frames}")
print(f"% of frames skipped: {(skipped_frames / frame_count) * 100:.2f}%")
print(f"Frames processing per second: {frame_count / processing_time:.2f} FPS")
print(f"Total time taken to load frames: {loading_time:.6f} seconds")
print(f"Total processing time: {processing_time:.6f} seconds")
print(f"Total time (loading + processing): {total_time:.6f} seconds")
print(f"Average time per frame comparison: {processing_time / (frame_count - 1):.6f} seconds")
print(f"Total memory usage: Initial = {initial_memory:.2f} MB, Final = {final_memory:.2f} MB")
print(f"Memory consumed during processing: {memory_consumed:.2f} MB")
print(f"Total processing load: {frame_count} frames")


Processed video saved as: /content/processed_video.mp4
Total number of frames: 1206
Number of frames skipped: 656
% of frames skipped: 54.39%
Frames processing per second: 172.42 FPS
Total time taken to load frames: 9.563020 seconds
Total processing time: 6.994671 seconds
Total time (loading + processing): 16.557690 seconds
Average time per frame comparison: 0.005805 seconds
Total memory usage: Initial = 1225.31 MB, Final = 1801.88 MB
Memory consumed during processing: 576.56 MB
Total processing load: 1206 frames
